In [12]:
import requests, os

bearer = "AAAAAAAAAAAAAAAAAAAAAHFl4wEAAAAAoJ2d0bhcr4oX7QlQEIgtL8Y1hUE%3Dq7NlqeVGzf4KlJMi3G1m942vegTjAiBVP01q8jGywCBcNzqAOB"   # replace with raw token (no % encoding, no quotes)
url = "https://api.twitter.com/2/users/by/username/OpenAI"

resp = requests.get(url, headers={"Authorization": f"Bearer {bearer}"})
print(resp.status_code)
print(resp.text)


429
{"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}


In [13]:
# token = "YOUR_BEARER_TOKEN"
print("len:", len(bearer))
print("contains %3D:", "%3D" in bearer)
print("first/last chars:", bearer[:4], bearer[-4:])


len: 112
contains %3D: True
first/last chars: AAAA qAOB


In [18]:
import os
import urllib.parse
import tweepy
from langchain.schema import Document

# Option A: read from environment (recommended)
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHFl4wEAAAAAoJ2d0bhcr4oX7QlQEIgtL8Y1hUE%3Dq7NlqeVGzf4KlJMi3G1m942vegTjAiBVP01q8jGywCBcNzqAOB"

# Option B: (only for quick local testing) uncomment and paste raw token:
# bearer_token = "AAAAAAAA...your_raw_bearer_token..."

# ---- sanity checks ----
if not bearer_token:
    raise SystemExit("ERROR: Bearer token not found. Set TWITTER_BEARER_TOKEN env var or assign bearer_token.")

# If token was accidentally URL-encoded (contains %3D etc), decode it:
if "%3D" in bearer_token or "%2F" in bearer_token:
    bearer_token = urllib.parse.unquote(bearer_token)

print("bearer token length:", len(bearer_token))
print("bearer token starts/ends:", bearer_token[:4], bearer_token[-4:])

# ---- Tweepy + LangChain usage ----
try:
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

    # quick sanity call (this should succeed with an app-only bearer token)
    me = client.get_me()
    print("Authenticated as:", me.data.username, me.data.id)

    # get user id for username
    user = client.get_user(username="OpenAI")
    user_id = user.data.id

    tweets = client.get_users_tweets(id=user_id, max_results=5)  # v2 endpoint
    docs = []
    if tweets and tweets.data:
        for t in tweets.data:
            docs.append(Document(page_content=t.text, metadata={"id": t.id, "author": user.data.username}))
    else:
        print("No tweets returned (could be protected account or nothing recent).")

    print("Loaded", len(docs), "documents")
except tweepy.errors.TweepyException as e:
    print("Tweepy error:", type(e).__name__, str(e))
except Exception as e:
    print("Error:", type(e).__name__, e)


bearer token length: 110
bearer token starts/ends: AAAA qAOB
Error: TypeError Consumer key must be string or bytes, not NoneType
